In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras_vggface.vggface import VGGFace


In [28]:
#custom parameters
nb_class = 5
hidden_dim = 512

vgg_model = VGGFace(model='vgg16',include_top=False, input_shape=(224, 224, 3))

In [29]:
#vgg_model.load_weights("C:\\Users\\isaac yang\\.keras\\models\\vggface\\rcmalli_vggface_tf_notop_vgg16.h5")

In [30]:
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [31]:
vgg_model.trainable = False

In [32]:
classifier_model=Sequential()


In [33]:
classifier_model.add(vgg_model)
classifier_model.add(Flatten())
# classifier_model.add(BatchNormalization())
# classifier_model.add(Activation('relu'))
# classifier_model.add(Dropout(0.3))
# classifier_model.add(Dense(units=10,kernel_initializer='glorot_uniform'))
# classifier_model.add(BatchNormalization())
# classifier_model.add(Activation('relu'))
# classifier_model.add(Dropout(0.2))
classifier_model.add(Dense(nb_class, activation = 'softmax'))


In [34]:
model = classifier_model

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vggface_vgg16 (Model)        (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 25088)             100352    
_________________________________________________________________
activation_7 (Activation)    (None, 25088)             0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 25088)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                250890    
_________________________________________________________________
batch_normalization_8 (Batch (None, 10)                40        
__________

In [35]:
from keras.preprocessing.image import ImageDataGenerator
img_width = img_height = 224
channels = 3
datagen = ImageDataGenerator(
    featurewise_center=True,
    zca_whitening=True,
    zca_epsilon=1e-06,
    #rotation_range=40, #I'm thinking if a person tilts his head it will be 40 degrees maximum
    #width_shift_range=0.5, #If up to half the person's face is visible width-wise
    #height_shift_range=0.5, #If up to half the person's face is visible height-wise
    #brightness_range=[0.2,2.0], #20% or 200% brightness
    #shear_range=10.0, #putting it very small here because I can't find any tutorials with exact values of good shear range angles
    #zoom_range=[0.3,3], #zooming up to 3 times and minimizing up to 3 times
    #channel_shift_range= 200, #Increases color variation in case someone's face suddenly becomes very tanned, can check the rgb values table
    fill_mode="nearest",
    horizontal_flip=True,
    rescale=1./255,
    validation_split=0.3,
)



In [36]:
batch_size = 16
train_it = datagen.flow_from_directory('data/train2/',
                                       target_size = (img_height, img_width),
                                       class_mode='categorical', 
                                       batch_size=batch_size)

val_it = datagen.flow_from_directory('data/val2/', 
                                    class_mode='categorical',
                                    target_size = (img_height, img_width),
                                    batch_size=batch_size)


Found 93 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


In [37]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
filepath="C:\\development\\NDSC_junior\\SRP\\weights\\vgg16\\weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"

# earlystop = EarlyStopping(monitor = 'val_loss',
#                          min_delta = 0,
#                          patience = 3,
#                          verbose = 1,
#                          restore_best_weights = True)
checkpoint = ModelCheckpoint(filepath,
                             monitor='val_loss', 
                             verbose=1,
                             save_best_only=True, 
                             mode='min')

nb_train_samples = 64
nb_validation_samples = 16

epochs = 5
steps_per_epoch = nb_train_samples/batch_size
validation_steps = nb_validation_samples/batch_size

from keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, min_delta = 0.0001, patience=3, min_lr=0, cooldown = 0)
callbacks = [checkpoint, reduce_lr]


model.compile(loss = 'categorical_crossentropy',
             optimizer = 'Adam',
             metrics = ['accuracy'])

In [38]:
model.fit_generator(train_it, 
                    validation_data = val_it, 
                    steps_per_epoch=steps_per_epoch, 
                    validation_steps=validation_steps, 
                    epochs = epochs, 
                    callbacks= callbacks)

Epoch 1/5
4/4 [==============================] - 28s 7s/step - loss: 1.6897 - acc: 0.2969 - val_loss: 0.9055 - val_acc: 0.6875

Epoch 00001: val_loss improved from inf to 0.90551, saving model to C:\development\NDSC_junior\SRP\weights\weights-improvement-01-0.91.hdf5
Epoch 2/5
4/4 [==============================] - 23s 6s/step - loss: 1.2014 - acc: 0.5667 - val_loss: 1.0700 - val_acc: 0.6667

Epoch 00002: val_loss did not improve from 0.90551
Epoch 3/5
4/4 [==============================] - 25s 6s/step - loss: 1.0038 - acc: 0.6973 - val_loss: 0.6858 - val_acc: 0.7500

Epoch 00003: val_loss improved from 0.90551 to 0.68584, saving model to C:\development\NDSC_junior\SRP\weights\weights-improvement-03-0.69.hdf5
Epoch 4/5
4/4 [==============================] - 24s 6s/step - loss: 0.8594 - acc: 0.7500 - val_loss: 0.6569 - val_acc: 0.8889

Epoch 00004: val_loss improved from 0.68584 to 0.65691, saving model to C:\development\NDSC_junior\SRP\weights\weights-improvement-04-0.66.hdf5
Epoch 5/5